In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd

In [2]:
df_states_original = pd.read_csv("./us-states.csv", dtype={"date": np.string_,
                                                             "state": np.string_,
                                                             "fips": np.string_,
                                                            "cases": np.int32,
                                                            "deaths": np.int32})
df_states_original.sort_values("fips", ascending=True)
# #Replaces any nan to 0
# if df_states_original.isnull().values.any():
#     df_states_original.replace(np.nan, None)

,date,state,fips,cases,deaths
4689,2020-05-27,Alabama,01,16032,583
1994,2020-04-08,Alabama,01,2498,67
3369,2020-05-03,Alabama,01,7888,290
1281,2020-03-26,Alabama,01,538,3
3919,2020-05-13,Alabama,01,10700,450
...,...,...,...,...,...
4738,2020-05-27,Virgin Islands,78,69,6
4298,2020-05-19,Virgin Islands,78,69,6
1221,2020-03-24,Virgin Islands,78,17,0
3308,2020-05-01,Virgin Islands,78,66,4


In [3]:
pivot_states = pd.pivot_table(df_states_original , index=['state','fips'],
                       columns=['date'])
if pivot_states.isnull().values.any():
    pivot_states = pivot_states.replace(np.nan, 0).astype(int)
pivot_states.head(5)

cases                                              \
date            2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state      fips                                                          
Alabama    01            0          0          0          0          0   
Alaska     02            0          0          0          0          0   
Arizona    04            0          0          0          0          0   
Arkansas   05            0          0          0          0          0   
California 06            0          0          0          0          1   

                                                                        ...  \
date            2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30  ...   
state      fips                                                         ...   
Alabama    01            0          0          0          0          0  ...   
Alaska     02            0          0          0          0          0  ...   
Arizona    04            1          1          1          1          1  ...   
Arkansas   05            0          0          0          0          0  ...   
California 06            2          2          2          2          2  ...   

                    deaths                                              \
date            2020-06-20 2020-06-21 2020-06-22 2020-06-23 2020-06-24   
state      fips                                                          
Alabama    01          838        839        841        864        891   
Alaska     02           10         10         10         10         10   
Arizona    04         1347       1350       1354       1398       1467   
Arkansas   05          224        225        227        237        240   
California 06         5495       5517       5561       5637       5728   

                                                                        
date            2020-06-25 2020-06-26 2020-06-27 2020-06-28 2020-06-29  
state      fips                                                         
Alabama    01          896        907        919        919        929  
Alaska     02           10         12         12         12         12  
Arizona    04         1495       1537       1581       1594       1598  
Arkansas   05          240        249        259        264        265  
California 06         5810       5872       5902       5937       5979  

[5 rows x 322 columns]

Get Date Info

In [4]:
date= pivot_states['cases'].columns
date

Index(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30',
       ...
       '2020-06-20', '2020-06-21', '2020-06-22', '2020-06-23', '2020-06-24',
       '2020-06-25', '2020-06-26', '2020-06-27', '2020-06-28', '2020-06-29'],
      dtype='object', name='date', length=161)

In [5]:
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

In [6]:
dt_start = datetime.strptime(dt_str_start, "%Y-%m-%d")
dt_end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=dt_start,end=dt_end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

161 DatetimeIndex(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24',
               '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28',
               '2020-01-29', '2020-01-30',
               ...
               '2020-06-20', '2020-06-21', '2020-06-22', '2020-06-23',
               '2020-06-24', '2020-06-25', '2020-06-26', '2020-06-27',
               '2020-06-28', '2020-06-29'],
              dtype='datetime64[ns]', length=161, freq='D')
161 ['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-

Add Cases Time Series, First Case Date, Death Time Series, First Death Date

In [7]:
# import json
# pivot_states['cases_ts'] = "'" + json.dumps({"values": pivot_states['cases'].values.tolist()[0]}) + "'"
# pivot_states['deaths_ts'] = "'" + json.dumps({"values": pivot_states['deaths'].values.tolist()[0]}) + "'"

pivot_states['cases_ts'] = pivot_states['cases'].values.tolist()
pivot_states['cases_ts'] = pivot_states['cases_ts'].apply(lambda x: ','.join(map(str, x)))
pivot_states['deaths_ts'] = pivot_states['deaths'].values.tolist()
pivot_states['deaths_ts'] = pivot_states['deaths_ts'].apply(lambda x: ','.join(map(str, x)))

In [8]:
yesterday = date.values[-2]
pivot_states['today_case'] = pivot_states['cases'][dt_str_end]
pivot_states['today_new_case'] = pivot_states['cases'][dt_str_end] - pivot_states['cases'][yesterday]
pivot_states['today_death'] = pivot_states['deaths'][dt_str_end]
pivot_states['today_new_death'] = pivot_states['deaths'][dt_str_end] - pivot_states['deaths'][yesterday]

In [9]:
pivot_states['dt_first_case'] = (pivot_states['cases'] > 0).idxmax(axis=1)
pivot_states['dt_first_death'] = (pivot_states['deaths'] > 0).idxmax(axis=1)
#For death reports, deal with no deaths counties
pivot_states.loc[pivot_states['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_states.head(5)

cases                                              \
date            2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state      fips                                                          
Alabama    01            0          0          0          0          0   
Alaska     02            0          0          0          0          0   
Arizona    04            0          0          0          0          0   
Arkansas   05            0          0          0          0          0   
California 06            0          0          0          0          1   

                                                                        ...  \
date            2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30  ...   
state      fips                                                         ...   
Alabama    01            0          0          0          0          0  ...   
Alaska     02            0          0          0          0          0  ...   
Arizona    04            1          1          1          1          1  ...   
Arkansas   05            0          0          0          0          0  ...   
California 06            2          2          2          2          2  ...   

                    deaths             \
date            2020-06-28 2020-06-29   
state      fips                         
Alabama    01          919        929   
Alaska     02           12         12   
Arizona    04         1594       1598   
Arkansas   05          264        265   
California 06         5937       5979   

                                                          cases_ts  \
date                                                                 
state      fips                                                      
Alabama    01    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
Alaska     02    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
Arizona    04    0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...   
Arkansas   05    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
California 06    0,0,0,0,1,2,2,2,2,2,3,3,6,6,6,6,6,6,6,6,7,7,7,...   

                                                         deaths_ts today_case  \
date                                                                            
state      fips                                                                 
Alabama    01    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...      37175   
Alaska     02    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...       1085   
Arizona    04    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...      74602   
Arkansas   05    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...      20257   
California 06    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...     223995   

                today_new_case today_death today_new_death dt_first_case  \
date                                                                       
state      fips                                                            
Alabama    01             1734         929              10    2020-03-13   
Alaska     02               25          12               0    2020-03-12   
Arizona    04              641        1598               4    2020-01-26   
Arkansas   05              439         265               1    2020-03-11   
California 06             7040        5979              42    2020-01-25   

                dt_first_death  
date                            
state      fips                 
Alabama    01       2020-03-25  
Alaska     02       2020-03-27  
Arizona    04       2020-03-20  
Arkansas   05       2020-03-24  
California 06       2020-03-04  

[5 rows x 330 columns]

In [10]:
pivot_states['today_case'].sum()

2604915

Read County Boundary GeoJSON file

In [11]:
old_states_geojson_df = gpd.read_file(r"./states_update.geojson")
old_states_geojson_df.head(5)

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,...,d2020-03-19,d2020-03-20,d2020-03-21,d2020-03-22,d2020-03-23,d2020-03-24,d2020-03-25,d2020-03-26,population,geometry
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,...,0,0,0,0,0,0,0,0,1829054,"POLYGON ((-80.51899 40.63880, -80.51934 39.721..."
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,...,1,1,2,2,4,3,0,6,20598139,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ..."
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,...,3,1,1,1,5,4,3,7,12821497,"POLYGON ((-90.64284 42.50848, -87.01994 42.493..."
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,...,0,0,1,0,0,0,0,1,5527358,"POLYGON ((-97.22872 49.00056, -95.15371 48.998..."
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,...,0,1,1,0,0,1,0,0,6003435,"POLYGON ((-77.71952 39.32131, -77.76086 39.340..."


In [12]:
old_states_geojson_df[['NAME','population','GEOID','geometry']]

,NAME,population,GEOID,geometry
0,West Virginia,1829054,54,"POLYGON ((-80.51899 40.63880, -80.51934 39.721..."
1,Florida,20598139,12,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ..."
2,Illinois,12821497,17,"POLYGON ((-90.64284 42.50848, -87.01994 42.493..."
3,Minnesota,5527358,27,"POLYGON ((-97.22872 49.00056, -95.15371 48.998..."
4,Maryland,6003435,24,"POLYGON ((-77.71952 39.32131, -77.76086 39.340..."
5,Rhode Island,1056611,44,"MULTIPOLYGON (((-71.67881 41.15891, -71.63497 ..."
6,Idaho,1687809,16,"POLYGON ((-116.91607 45.99536, -116.98259 46.0..."
7,New Hampshire,1343622,33,"POLYGON ((-71.50108 45.01335, -71.49804 45.070..."
8,North Carolina,10155624,37,"POLYGON ((-81.67749 36.58814, -80.12217 36.542..."
9,Vermont,624977,50,"POLYGON ((-73.34312 45.01084, -71.46505 45.013..."


Old Data Structure

In [13]:
report_df = pivot_states[['cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']]
report_df = report_df.reset_index()
report_df.columns = ['state','fips','cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']
report_df.head(5)

,state,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Alabama,01,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-03-13,2020-03-25,37175,1734,929,10
1,Alaska,02,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-03-12,2020-03-27,1085,25,12,0
2,Arizona,04,"0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-01-26,2020-03-20,74602,641,1598,4
3,Arkansas,05,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-03-11,2020-03-24,20257,439,265,1
4,California,06,"0,0,0,0,1,2,2,2,2,2,3,3,6,6,6,6,6,6,6,6,7,7,7,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-01-25,2020-03-04,223995,7040,5979,42


In [14]:
report_df['state']

0                      Alabama
1                       Alaska
2                      Arizona
3                     Arkansas
4                   California
5                     Colorado
6                  Connecticut
7                     Delaware
8         District of Columbia
9                      Florida
10                     Georgia
11                        Guam
12                      Hawaii
13                       Idaho
14                    Illinois
15                     Indiana
16                        Iowa
17                      Kansas
18                    Kentucky
19                   Louisiana
20                       Maine
21                    Maryland
22               Massachusetts
23                    Michigan
24                   Minnesota
25                 Mississippi
26                    Missouri
27                     Montana
28                    Nebraska
29                      Nevada
30               New Hampshire
31                  New Jersey
32      

In [15]:
report_df['today_case'].sum()
#truth value: 528405

2604915

In [16]:
#Stay consistency with geojson states
report_df = report_df.replace('Northern Mariana Islands','Commonwealth of the Northern Mariana Islands')

In [17]:
final_df = pd.merge(old_states_geojson_df[["NAME", "population", "geometry"]], report_df, how='left', left_on=['NAME'], right_on = ['state'])
final_df.columns

Index(['NAME', 'population', 'geometry', 'state', 'fips', 'cases_ts',
       'deaths_ts', 'dt_first_case', 'dt_first_death', 'today_case',
       'today_new_case', 'today_death', 'today_new_death'],
      dtype='object')

In [18]:
final_df['NAME']

0                                    West Virginia
1                                          Florida
2                                         Illinois
3                                        Minnesota
4                                         Maryland
5                                     Rhode Island
6                                            Idaho
7                                    New Hampshire
8                                   North Carolina
9                                          Vermont
10                                     Connecticut
11                                        Delaware
12                                      New Mexico
13                                      California
14                                      New Jersey
15                                       Wisconsin
16                                          Oregon
17                                        Nebraska
18                                    Pennsylvania
19                             

In [19]:
final_df=final_df[["NAME", "population", "fips", "dt_first_case", "dt_first_death", "cases_ts", "deaths_ts", 'today_case','today_new_case', 'today_death','today_new_death', "geometry"]]
final_df['dt_start'] = dt_str_start
final_df['dt_end'] = dt_str_end
final_df['dt_unit'] = "day"
final_df.head(5)

,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,dt_start,dt_end,dt_unit
0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2870.0,38.0,93.0,0.0,"POLYGON ((-80.51899 40.63880, -80.51934 39.721...",2020-01-21,2020-06-29,day
1,Florida,20598139,12,2020-03-01,2020-03-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",146333.0,5266.0,3446.0,28.0,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ...",2020-01-21,2020-06-29,day
2,Illinois,12821497,17,2020-01-24,2020-03-17,"0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",143885.0,768.0,7115.0,17.0,"POLYGON ((-90.64284 42.50848, -87.01994 42.493...",2020-01-21,2020-06-29,day
3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",35896.0,312.0,1470.0,10.0,"POLYGON ((-97.22872 49.00056, -95.15371 48.998...",2020-01-21,2020-06-29,day
4,Maryland,6003435,24,2020-03-05,2020-03-18,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",67837.0,477.0,3175.0,7.0,"POLYGON ((-77.71952 39.32131, -77.76086 39.340...",2020-01-21,2020-06-29,day


In [20]:
final_df['today_case'].sum()
# result: 528405, same as truth

2604915.0

In [21]:
#New York Times remove Samoa data since 4/27
final_df = final_df.drop(final_df[final_df['NAME'] == 'American Samoa'].index)

In [22]:
final_df.to_file(r"./nyt_states_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

done


New Data Structure (Full version)

In [23]:
# pivot_states.columns = ['_'.join(col).strip() for col in pivot_states.columns.values]
# report_df_2 = pivot_states.reset_index()

In [24]:
# report_df_2.head(5)

In [25]:
# pivot_states.head(5)

In [26]:
# final_df_2 = pd.merge(old_states_geojson_df[["NAME", "population", "geometry"]], report_df_2, how='right', left_on=['NAME'], right_on = ['state'])
# final_df_2 = final_df_2.drop(['state'], axis = 1)
# final_df_2['dt_start'] = dt_start
# final_df_2['dt_end'] = dt_end
# final_df_2.head(5)

In [27]:
# final_df_2.to_file(r"./nyt_states_data_full.geojson", driver='GeoJSON', encoding='utf-8')
# print("done")

